In [5]:
import wget

wget.download(
    url="https://github.com/KLUE-benchmark/KLUE/raw/main/klue_benchmark/klue-mrc-v1.1/klue-mrc-v1.1_train.json",
    out="./data/klue-mrc-v1.1_train.json",
)

'klue-mrc-v1.1_train.json'

In [16]:
import json
import pandas as pd
from tqdm import tqdm
from FlagEmbedding import BGEM3FlagModel
import numpy as np
import datasets
from huggingface_hub import login
from datasets import load_dataset
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
with open("klue-mrc-v1.1_train.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 필요한 정보 추출
records = []
for article in data["data"]:
    title = article["title"]
    news_category = article.get("news_category", "N/A")
    source = article.get("source", "N/A")

    for paragraph in article["paragraphs"]:
        context = paragraph["context"]

        for qa in paragraph["qas"]:
            question = qa["question"]
            question_type = qa["question_type"]
            is_impossible = qa["is_impossible"]

            for answer in qa["answers"]:
                answer_text = answer["text"]
                answer_start = answer["answer_start"]

                records.append(
                    {
                        "title": title,
                        "news_category": news_category,
                        "source": source,
                        "context": context,
                        "question": question,
                        "question_type": question_type,
                        "is_impossible": is_impossible,
                        "answer_text": answer_text,
                        "answer_start": answer_start,
                    }
                )

# DataFrame으로 변환
df = pd.DataFrame(records)

In [7]:
# 질문 기준으로 중복 샘플 제거
df = df.drop_duplicates(subset="question")

In [8]:
# 본문 기준으로 중복 샘플 제거
df = df.drop_duplicates(subset="context")

In [9]:
# 질문, 본문, 정답
df[["context", "question", "answer_text"]]

,context,question,answer_text
0,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,한 달가량
2,부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,지능형 생산자동화 기반기술을 개발중인 스타트업은?,삼보테크놀로지
3,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?,와쿠이 히데아키
4,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,컵라면 매출에서 불닭볶음면을 이긴 상품은?,‘교동반점 짬뽕’
8,“앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...,정부에게 환경과 관련해서 우선적으로 원조 받고 있는 곳은?,환경오염이 심한 지역
...,...,...,...
17656,“오너 경영은 중견기업이 글로벌 전문기업이나 대기업으로 성장하는 밑거름이 될 것입니...,이동기 학회장은 오너 경영을 무엇과 비교했는가?,전문 경영인 체제
17657,홈 뷰티 디바이스 브랜드 ‘엘리닉(L.linic)’이 고객 감사의 달을 맞아 전국 ...,팝업스토어 앵콜 행사를 가장 먼저 시작하는 곳은 어디인가?,롯데백화점 잠실점
17658,유인원 무리의 리더 시저는 인간 건축가 말콤(제이슨 클락)에게 작별 인사를 한다. ...,혹성탈출의 두 번째 프리퀄의 제목은?,‘혹성탈출: 반격의 서막’
17660,ASUS(에이수스) 그래픽카드 공식수입사 인텍앤컴퍼니(대표 서정욱)는 10월 16일...,인택엔컴퍼니가 실시하는 추첨판매 신청 마감시간은?,8시 10분


In [12]:
# 임베딩 모델 로드
model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)

c:\Users\Songjae\AppData\Local\pypoetry\Cache\virtualenvs\raft-rag-2xzNYwoc-py3.11\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Songjae\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 30 files: 100%|██████████| 30/30 [01:15<00:00,  2.51s/it]


In [13]:
# 문장을 임베딩으로 변환하는 함수
def get_embeddings(sentences, batch_size=32):
    return model.encode(sentences, batch_size=batch_size, max_length=512)["dense_vecs"]

In [14]:
# 최대 32개의 데이터를 동시에 임베딩하는 함수. 훨씬 빠르게 임베딩하기 위해 배치(데이터 묶음)로 처리합니다.
embeddings = get_embeddings(["안녕하세요", "반갑습니다"])
print("임베딩 벡터의 개수:", len(embeddings))

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


임베딩 벡터의 개수: 2


In [15]:
# 본문 전체를 임베딩
print("Generating context embeddings...")
context_embeddings = get_embeddings(df["context"].tolist())

Generating context embeddings...


Inference Embeddings:   2%|▏         | 6/327 [03:22<3:00:48, 33.80s/it]


KeyboardInterrupt: 

In [ ]:
# negative_samples 열을 추가할 리스트
negative_samples_list = []

# 각 질문(question)에 대해 유사도가 높은 4개의 다른 본문(context)를 찾기
print("Finding negative samples...")
for i in tqdm(range(len(df)), total=len(df), dynamic_ncols=True):
    question = df.iloc[i]["question"]

    # 질문 임베딩 생성
    question_embedding = get_embeddings([question])[0]

    # 코사인 유사도 계산
    similarities = np.dot(context_embeddings, question_embedding)

    # 현재 문서의 인덱스를 제외한 4개의 문서를 선택
    top_n_indices = similarities.argsort()[::-1]
    top_n_indices = [idx for idx in top_n_indices if idx != i][
        :4
    ]  # 현재 문서 제외, 상위 4개 선택

    # 선택된 문서들을 negative_samples 리스트로 추가
    negative_samples = [df["context"].iloc[idx] for idx in top_n_indices]
    negative_samples_list.append(negative_samples)

In [ ]:
# 결과를 새로운 열로 추가. 각 질문에 대해서 전체 본문(10,434개)과 유사도 비교를 했을때 유사도가 높은 본문 4개가 추가된다.
df["negative_samples"] = negative_samples_list

# 결과 확인
print(df.head())  # 결과의 처음 5개 행을 출력

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
idx = 5

print("질문 :", df["question"].loc[idx])
print("정답 문서:", df["context"].loc[idx])
print("네거티브 문서:", df["negative_samples"].loc[idx])
print("네거티브 문서의 개수:", len(df["negative_samples"].loc[idx]))

In [ ]:
# API 토큰으로 로그인 (발급받은 토큰을 입력)
# 허깅페이스 홈페이지에 로그인 후에 'Acess Tokens' 메뉴를 눌러서 발급받을 수 있습니다.
# 키는 주로 hf로 시작하며 이 Key를 통해서 허깅페이스에 여러분들의 모델이나 데이터셋을 업로드 할 수 있습니다.
# 저는 제 계정 iamjoon에 데이터셋을 올리기 위해서 사용했습니다.
login("여러분들의 허깅페이스 Key")

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset.push_to_hub("klue-mrc-bge-m3")

In [ ]:
# 1. Hugging Face에서 데이터셋 불러오기
dataset = load_dataset("longjae/klue-mrc-bge-m3")

# 2. train split을 데이터프레임으로 변환
df = dataset["train"].to_pandas()

# 3. 데이터프레임 확인
print(df.head())

In [ ]:
df